### Data Dictionary
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

In [20]:
# ∗∗∗∗∗∗∗∗∗∗ Important – Do not use in production, for demonstration purposes only – please review the legal notices before continuing ∗∗∗∗∗

from pyspark.sql import SparkSession
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from azure.storage.blob import ContainerClient, BlobClient
import pandas as pd
from io import BytesIO
from copy import deepcopy

StatementMeta(RetailPool, 9, 1, Finished, Available)

In [21]:
CONNECTIONSTRING = 'DefaultEndpointsProtocol=https;AccountName=#STORAGE_ACCOUNT_NAME#;AccountKey=#STORAGE_ACCOUNT_KEY#;EndpointSuffix=core.windows.net'
CONTAINER_NAME = 'customer-churn-data'

BLOBNAME = 'online_retail_II.xlsx'
blob = BlobClient.from_connection_string(conn_str=CONNECTIONSTRING, container_name=CONTAINER_NAME, blob_name=BLOBNAME)
blob_data = blob.download_blob()
BytesIO(blob_data.content_as_bytes())
data = pd.read_excel(BytesIO(blob_data.content_as_bytes()))

StatementMeta(RetailPool, 9, 2, Finished, Available)

In [22]:
data.head()

StatementMeta(RetailPool, 9, 3, Finished, Available)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [23]:
# data.describe(include = 'all')

StatementMeta(RetailPool, 9, 4, Finished, Available)

In [24]:
data.shape

StatementMeta(RetailPool, 9, 5, Finished, Available)

(525461, 8)

In [25]:
data.dtypes

StatementMeta(RetailPool, 9, 6, Finished, Available)

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [26]:
data.dropna(inplace=True)

StatementMeta(RetailPool, 9, 7, Finished, Available)

In [27]:
data.isnull().sum() 

StatementMeta(RetailPool, 9, 8, Finished, Available)

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [28]:
data.describe()

StatementMeta(RetailPool, 9, 9, Finished, Available)

,Quantity,Price,Customer ID
count,417534.000000,417534.000000,417534.000000
mean,12.758815,3.887547,15360.645478
std,101.220424,71.131797,1680.811316
min,-9360.000000,0.000000,12346.000000
25%,2.000000,1.250000,13983.000000
50%,4.000000,1.950000,15311.000000
75%,12.000000,3.750000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [29]:
# Convert InvocieDate to date.
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

StatementMeta(RetailPool, 9, 10, Finished, Available)

In [38]:
df_cancel = data[data["Invoice"].str.contains("C",na = False)]

StatementMeta(RetailPool, 9, 19, Finished, Available)

In [43]:
df_cancel.groupby("Invoice").agg(Invoice_Count=('Invoice', 'count')).sort_values("Invoice_Count", ascending=False).head(5)

StatementMeta(RetailPool, 9, 24, Finished, Available)

,Invoice_Count
Invoice,
C524235,45
C536164,44
C509015,40
C531807,38
C512272,38


In [47]:
data[data['Invoice'] == 'C524235'].shape

StatementMeta(RetailPool, 9, 28, Finished, Available)

(45, 8)